In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier # <-- New Model Import
from sklearn.metrics import f1_score

In [2]:
# --- A. LOAD DATA AND SPLIT ---
try:
    # Load your encoded and ready-to-scale features
    df_train = pd.read_csv('data/train_processed.csv')
    df_test = pd.read_csv('data/test_processed.csv')
    test_ids = pd.read_csv('data/ids.csv')
except FileNotFoundError:
    print("FATAL ERROR: Could not find files. Please check file paths ('data/').")
    raise

# Define X/y
X_train_full = df_train.drop(['has_copd_risk'], axis=1, errors='ignore')
y_train_full = df_train['has_copd_risk']

# Split for Preprocessor fitting (80% Train, 20% Val)
X_train, X_val, y_train, y_val = train_test_split(
    X_train_full, y_train_full, test_size=0.2, random_state=42, stratify=y_train_full
)

# --- B. PREPROCESSING PIPELINE (Fit/Transform) ---
# The standard scaling is CRITICAL for KNN.
all_features = X_train.columns.tolist()

numerical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()) 
])

preprocessor = ColumnTransformer(
    transformers=[('num', numerical_pipeline, all_features)],
    remainder='passthrough'
)

# Fit preprocessor on training data and transform it
X_train_processed = preprocessor.fit_transform(X_train)

# --- C. TRAIN MODEL, PREDICT, AND SAVE SUBMISSION ---

# 1. Train K-Nearest Neighbors (KNN) Model
# n_neighbors=5 is a common starting point
knn_model = KNeighborsClassifier(n_neighbors=5, n_jobs=-1) 
knn_model.fit(X_train_processed, y_train)
print("✅ K-Nearest Neighbors (KNN) Model trained successfully.")

# 2. Evaluate Baseline F1 Score on Validation Set
X_val_processed = preprocessor.transform(X_val)
y_pred_val = knn_model.predict(X_val_processed)
f1 = f1_score(y_val, y_pred_val)
print(f"Validation F1 Score (Baseline): {f1:.4f}")

# 3. Prepare Test Features and Predict
X_test_full = df_test.drop('patient_id', axis=1, errors='ignore')
X_test_full = X_test_full[all_features] 

# Apply Preprocessor to Test Data (Transform only!)
X_test_processed = preprocessor.transform(X_test_full)

# Prediction
y_test_pred = knn_model.predict(X_test_processed)

# 4. Combine IDs and Predictions and Save
submission_df = test_ids.copy()
submission_df['has_copd_risk'] = y_test_pred.astype(int)

submission_df.to_csv('submission/knn.csv', index=False)
print("\n✅ KNN submission file saved to 'submission/knn.csv'")
print("\nFirst 5 predictions in submission:")
print(submission_df.head())

✅ K-Nearest Neighbors (KNN) Model trained successfully.
Validation F1 Score (Baseline): 0.6187

✅ KNN submission file saved to 'submission/knn.csv'

First 5 predictions in submission:
   patient_id  has_copd_risk
0       42427              0
1       27412              0
2       19283              1
3       45261              0
4       11155              0
